In [1]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [ ]:
# !pip install py7zr

In [2]:
!pip install ipympl
!pip install matplotlib --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 KB 31.5 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3


In [ ]:
# !py7zr x /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/FBX_Data_V2.7z
# !py7zr x /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders.7z

In [ ]:
# !cp -r /content/FBX_Data_V2 /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/
# !cp -r /content/Renders /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/

In [ ]:
# !cp -r /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/FBX_Data_V2 /content
# !cp -r /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders /content

In [3]:
import gc
import os
from shutil import copy2,rmtree
import json
from tqdm import tqdm
import pickle
import random;random.seed(53)
from datetime import datetime

from functools import partial


import numpy as np
import cv2
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
# try:
#   device_name = os.environ['COLAB_TPU_ADDR']
#   TPU_ADDRESS = 'grpc://' + device_name
#   print('Found TPU at: {}'.format(TPU_ADDRESS))
#   tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
#   tf.config.experimental_connect_to_cluster(tpu_resolver)
#   tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
#   tpu_strategy = tf.distribute.TPUStrategy(tpu_resolver)
# except KeyError:
#   print('TPU not found')

Found TPU at: grpc://10.85.46.106:8470


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
joints = [
         "Shoulder_L","Elbow_L","Wrist_L",
         "Shoulder_R","Elbow_R","Wrist_R",
         "Pelvis_L","Knee_L","Ankle_L",
         "Pelvis_R","Knee_R","Ankle_R"
]

In [ ]:
label_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Render_Labels"
files = os.listdir(label_dir)

labels = {}

for i in files:
    keypoints = []
    with open(os.path.join(label_dir,i),"r") as a:
        dic = json.load(a)
        
                
    dic = list(dic.values())
        
    for j in range(len(dic)):
        temp = []
        for k in joints:
            te = dic[j][k]
            for l in te:
                temp.append(l/100)
        # print(temp)
        keypoints.append(temp)
        
    keypoints_scaled = scaler.fit_transform(keypoints)
    labels[i[:-5]] = keypoints_scaled
            
len(labels)

16

In [ ]:
print(labels.keys())
print(len(list(labels.values())[0]))
print(len(list(labels.values())[0][0]))

dict_keys(['tmp_2', 'tmp_1_r', 'tmp_6', 'tmp_10', 'tmp_7', 'tmp_8', 'tmp_12', 'tmp_2_r', 'tmp_1', 'tmp_13', 'tmp_3', 'tmp_4', 'tmp_6_r', 'tmp_11', 'tmp_9', 'tmp_5'])
1910
36


---

In [ ]:
labels["tmp_1_r"]

array([[0.70040547, 0.87805002, 0.54813887, ..., 0.36184861, 0.0227302 ,
        0.50591899],
       [0.6984425 , 0.86112908, 0.64917407, ..., 0.35647189, 0.05128916,
        0.53203595],
       [0.68990915, 0.8234097 , 0.71384599, ..., 0.35111159, 0.11000309,
        0.55016192],
       ...,
       [0.58613076, 0.91408958, 0.61461221, ..., 0.45938682, 0.01794096,
        0.43772506],
       [0.66983429, 0.9047077 , 0.54746941, ..., 0.49384482, 0.02106799,
        0.41630686],
       [0.68108881, 0.89993289, 0.52001651, ..., 0.48574019, 0.0174662 ,
        0.44870642]])

In [ ]:
print(len(os.listdir(r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders")))
print(len(os.listdir(r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Renders")))

16


In [ ]:
# main_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders"
# main_files = os.listdir(main_dir)

# for i in main_files:
#   for j in os.listdir(os.path.join(main_dir,i)):
#     if "Render_test" in j:
#       os.rename(os.path.join(main_dir,i,j),os.path.join(main_dir,i,j.replace("Render_test","")))

## Rename Images

In [ ]:
# main_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders"

# for dirs,progress in zip(os.listdir(main_dir),tqdm(range(len(os.listdir(main_dir))),desc=f"Renaming...")):
#     for i in os.listdir(os.path.join(main_dir,dirs)):
#         if "tmp" not in i:
#             os.rename(os.path.join(main_dir,dirs,i),os.path.join(main_dir,dirs,f"{dirs}_{i}"))

---

## Copy all images and preprocessing them to  Folder

In [ ]:
# try:
#   try:
#     rmtree("/content/All_Renders")
#     print("Removed All_Renders")
#   except:
#     os.mkdir("/content/All_Renders")
#     print("Created All_Renders")
# except:
#   pass

# main_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders"
# dest_dir = r"/content/All_Renders"

# for dirs,progress in zip(os.listdir(main_dir),tqdm(range(len(os.listdir(main_dir))),desc=f"Copying...")):
#     for i in os.listdir(os.path.join(main_dir,dirs)):
#         copy2(os.path.join(main_dir,dirs,i),os.path.join(dest_dir,i))
        
#         img = Image.open(os.path.join(dest_dir,i))
#         bg = Image.new("RGB",img.size,(255,255,255))
#         # bg = Image.open(os.path.join(backgrounds,f"{random.randint(0,len(bg_list)-1)}.jpg")).resize((200,200))
#         bg.paste(img,mask=img)
#         # bg = bg.convert("L")
#         bg.save(os.path.join(dest_dir,i.replace("png","jpg")))
#         os.remove(os.path.join(dest_dir,i))

# try:
#   rmtree("/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Renders")
#   print("Removed All_Renders")
# except:
#   pass

# !cp -r /content/All_Renders  /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data  

In [ ]:
len(os.listdir("/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders/tmp_2"))

449

---

## All Images and Keypoints to Dataframe

In [ ]:
keypoint_name = []

for i in range(0,len(keypoints[0])):
    keypoint_name.append(f"keypoints_{i}")

keypoint_name

In [ ]:
# data = pd.DataFrame(columns=["images"]+keypoint_name)

# images_dir =r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Renders"

# keypoint_count = 0

# for file,progress,index in zip(os.listdir(images_dir),tqdm(range(len(os.listdir(images_dir))),desc="Adding to DataFrame"),range(0,len(os.listdir(images_dir)))):
#     if keypoint_count != 1910:
#         data.loc[index] = os.path.join(images_dir,file)
#         for i in range(0,len(keypoints[0])):
#             data[keypoint_name[i]][index] = labels[file[:-9]][keypoint_count][i]
#         keypoint_count += 1

#     else:
#         keypoint_count = 0
#         data.loc[index] = os.path.join(images_dir,file)
#         for i in range(0,len(keypoints[0])):
#             data[keypoint_name[i]][index] = labels[file[:-9]][keypoint_count][i]
#         keypoint_count += 1

    
# data.to_csv(r"/content/All_Images_Dataset.csv",index=False)  

# !cp /content/All_Images_Dataset.csv /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data      

# data

In [ ]:
# !cp /content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Images_Dataset.csv /content

images_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Renders"
data = pd.read_csv(r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/All_Images_Dataset.csv",index_col=False)
data = shuffle(data,random_state=42)
data

---

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data["images"],data[keypoint_name],test_size=0.1)

train_df = pd.DataFrame(columns=["images"]+keypoint_name)
train_df["images"] = x_train
train_df[keypoint_name] = y_train[keypoint_name]

test_df = pd.DataFrame(columns=["images"]+keypoint_name)
test_df["images"] = x_test
test_df[keypoint_name] = y_test[keypoint_name]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255) 
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df, 
                                              x_col="images", y_col=keypoint_name, has_ext=True, 
                                              class_mode="multi_output", target_size=(200, 200), 
                                              batch_size=32)

test_datagen = ImageDataGenerator(rescale = 1./255) 
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, 
                                              x_col="images", y_col=keypoint_name, has_ext=True, 
                                              class_mode="multi_output", target_size=(200, 200), 
                                              batch_size=32)

In [ ]:
# backgrounds = "./Background/"
# bg_list = os.listdir(backgrounds)

In [ ]:
dirs = []
main_dir = r"/content/drive/MyDrive/Blender_Build/Renders_and_FBX_Data/Renders"

for i in os.listdir(main_dir):
    dirs.append(os.path.join(main_dir,i))
dirs

## Preprocessing

In [ ]:
train_count = 1
items_left = len(dirs)


def image_preprocess(batch_size,test_size=0.2,input_size=(200,200)):
    x_train = []
    sets = 1
    global items_left
    
    from_ = (train_count*batch_size)-batch_size
    to_ = train_count*batch_size
    
    for j in dirs[from_:to_]:
        for i,t in zip(os.listdir(j),tqdm(range(len(os.listdir(j))),desc=f"Loading... ===> Set-{sets}/{batch_size}")):
            img = Image.open(os.path.join(j,i)).resize((input_size[0],input_size[1]))
            bg = Image.new("RGB",img.size,(255,255,255))
            # bg = Image.open(os.path.join(backgrounds,f"{random.randint(0,len(bg_list)-1)}.jpg")).resize((200,200))
            bg.paste(img,mask=img)
            # bg = bg.convert("L")
            # img = np.asarray(bg).flatten()/255
            img = np.asarray(bg)/255
            x_train.append(img)
        
        sets += 1

            
    x_train = np.array(x_train)   
    print(f"x_train ----->  {x_train.shape}") 
    
    y_train = np.array(keypoints)
    y_train = scaler.inverse_transform(y_train) 
    print(f"y_train ----->  {y_train.shape}")
    

        
    
    x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=test_size,random_state=42,shuffle=True)
    print(f"x_train,x_test,y_train,y_test ----split---->  {x_train.shape,x_test.shape,y_train.shape,y_test.shape}")
    
    #-----------cleaning---------------
    del img,sets,from_,to_
    #-----------cleaning---------------
    
    return x_train,x_test,y_train,y_test
    

---

## Plotting

In [ ]:
colors = ["red","blue","green","purple","orange"]

# ====================================================

s2 = [
    "Shoulder_L","Elbow_L","Wrist_L",
]

s3 = [
    "Shoulder_R","Elbow_R","Wrist_R",
]

s4 = [
    "Pelvis_L","Knee_L","Ankle_L",
]

s5 = [
    "Pelvis_R","Knee_R","Ankle_R"
]

s6_connect = [
    "Shoulder_L",
    "Pelvis_L",
    "Pelvis_R",
    "Shoulder_R",
]

# ====================================================


def empty_coordinate():
    main_coordinates = {
        "s1" : {
            "x" : [],
            "y" : [],
            "z" : []
        },

        "s2" : {
            "x" : [],
            "y" : [],
            "z" : []
        },

        "s3" : {
            "x" : [],
            "y" : [],
            "z" : []
        },

        "s4" : {
            "x" : [],
            "y" : [],
            "z" : []
        },

        "s5" : {
            "x" : [],
            "y" : [],
            "z" : []
        },
        
        "s6_connect" : {
            "0" : {
                "x" : [],
                "y" : [],
                "z" : []
        },
            
            "1" : {
                "x" : [],
                "y" : [],
                "z" : []
        },
            
            "2" : {
                "x" : [],
                "y" : [],
                "z" : []
        },
            
            "3" : {
                "x" : [],
                "y" : [],
                "z" : []
        }
        }
        
    }
    
    return main_coordinates

In [ ]:
%matplotlib widget
def plot_results(data):
    coordinates = data
    main_coordinates = empty_coordinate()
    
    for i in list(coordinates.keys()):
        # if i in s1:
        #     x,y,z = coordinates[i]
        #     main_coordinates["s1"]["x"].append(x)
        #     main_coordinates["s1"]["y"].append(y)
        #     main_coordinates["s1"]["z"].append(z)
            
        if i in s2:
            x,y,z = coordinates[i]
            main_coordinates["s2"]["x"].append(x)
            main_coordinates["s2"]["y"].append(y)
            main_coordinates["s2"]["z"].append(z)
            
        elif i in s3:
            x,y,z = coordinates[i]
            main_coordinates["s3"]["x"].append(x)
            main_coordinates["s3"]["y"].append(y)
            main_coordinates["s3"]["z"].append(z)
            
        elif i in s4:
            x,y,z = coordinates[i]
            main_coordinates["s4"]["x"].append(x)
            main_coordinates["s4"]["y"].append(y)
            main_coordinates["s4"]["z"].append(z)
            
        elif i in s5:
            x,y,z = coordinates[i]
            main_coordinates["s5"]["x"].append(x)
            main_coordinates["s5"]["y"].append(y)
            main_coordinates["s5"]["z"].append(z)
            

    count = 0
    for i in range(len(s6_connect)):
        if count != 3:
            
            x,y,z = coordinates[s6_connect[i]]
            main_coordinates["s6_connect"][str(count)]["x"].append(x)
            main_coordinates["s6_connect"][str(count)]["y"].append(y)
            main_coordinates["s6_connect"][str(count)]["z"].append(z)
            
            x,y,z = coordinates[s6_connect[i+1]]
            main_coordinates["s6_connect"][str(count)]["x"].append(x)
            main_coordinates["s6_connect"][str(count)]["y"].append(y)
            main_coordinates["s6_connect"][str(count)]["z"].append(z)
            count += 1
        else:        
            
            x,y,z = coordinates[s6_connect[i]]
            main_coordinates["s6_connect"][str(count)]["x"].append(x)
            main_coordinates["s6_connect"][str(count)]["y"].append(y)
            main_coordinates["s6_connect"][str(count)]["z"].append(z)
            
            x,y,z = coordinates[s6_connect[0]]
            main_coordinates["s6_connect"][str(count)]["x"].append(x)
            main_coordinates["s6_connect"][str(count)]["y"].append(y)
            main_coordinates["s6_connect"][str(count)]["z"].append(z)
            
    # ====================================================

    plt.rcParams["figure.figsize"] = [7, 7]
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")

    # ax.scatter(main_coordinates["s1"]["x"], main_coordinates["s1"]["y"], main_coordinates["s1"]["z"], c=colors[0], s=100)
    ax.scatter(main_coordinates["s2"]["x"], main_coordinates["s2"]["z"], main_coordinates["s2"]["y"], c=colors[1], s=10)
    ax.scatter(main_coordinates["s3"]["x"], main_coordinates["s3"]["z"], main_coordinates["s3"]["y"], c=colors[2], s=10)
    ax.scatter(main_coordinates["s4"]["x"], main_coordinates["s4"]["z"], main_coordinates["s4"]["y"], c=colors[3], s=10)
    ax.scatter(main_coordinates["s5"]["x"], main_coordinates["s5"]["z"], main_coordinates["s5"]["y"], c=colors[4], s=10)


    # ax.plot(main_coordinates["s1"]["x"], main_coordinates["s1"]["y"], main_coordinates["s1"]["z"], color=colors[0])
    ax.plot(main_coordinates["s2"]["x"], main_coordinates["s2"]["z"], main_coordinates["s2"]["y"], color=colors[1])
    ax.plot(main_coordinates["s3"]["x"], main_coordinates["s3"]["z"], main_coordinates["s3"]["y"], color=colors[2])
    ax.plot(main_coordinates["s4"]["x"], main_coordinates["s4"]["z"], main_coordinates["s4"]["y"], color=colors[3])
    ax.plot(main_coordinates["s5"]["x"], main_coordinates["s5"]["z"], main_coordinates["s5"]["y"], color=colors[4])

    ax.plot(main_coordinates["s6_connect"]["0"]["x"], main_coordinates["s6_connect"]["0"]["z"], main_coordinates["s6_connect"]["0"]["y"], color="black")
    ax.plot(main_coordinates["s6_connect"]["1"]["x"], main_coordinates["s6_connect"]["1"]["z"], main_coordinates["s6_connect"]["1"]["y"], color="black")
    ax.plot(main_coordinates["s6_connect"]["2"]["x"], main_coordinates["s6_connect"]["2"]["z"], main_coordinates["s6_connect"]["2"]["y"], color="black")
    ax.plot(main_coordinates["s6_connect"]["3"]["x"], main_coordinates["s6_connect"]["3"]["z"], main_coordinates["s6_connect"]["3"]["y"], color="black")

    ax.set_xlabel('X-Axis')
    ax.set_ylabel('Y-Axis')
    ax.set_zlabel('Z-Axis')
    

    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()            

---

## Model Training

In [ ]:
try:
    rmtree(r"/content/Logs/train", ignore_errors=True)
    rmtree(r"/content/Logs/validation", ignore_errors=True)
except:
    print('Some Error Occured while deleting the folders :( .....')

In [ ]:
class MPJPELoss(tf.keras.losses.Loss):
    def __init__(self, name='mpjpe_loss'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        # Compute the Euclidean distance between the predicted and ground truth joint positions
        diff = y_pred - y_true
        dist = tf.norm(diff, axis=-1)
        
        # Compute the mean per joint position error (MPJPE)
        mpjpe = tf.reduce_mean(dist)
        return mpjpe

In [ ]:
model = tf.keras.Sequential([
    # First convolutional layer
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    
    # Second convolutional layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    
    # Third convolutional layer
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    
    # Flatten the output of the convolutional layers
    tf.keras.layers.Flatten(),
    
    # First fully connected layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Second fully connected layer
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer
    tf.keras.layers.Dense(36, activation='linear')
])


In [ ]:
try:
  os.mkdir(r"/content/Models")
except:
  pass

In [ ]:
# model.summary()

model.compile(optimizer='adam', loss=MPJPELoss())

# Print the model summary
model.summary()

In [ ]:
Log_callback = tf.keras.callbacks.TensorBoard(log_dir='./Logs/', histogram_freq=0,
                          write_graph=True, write_images=False)

Model_Checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./Models/model_{epoch:02d}_{val_loss:.4f}.h5")

Earlystop_callback = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=2,
                    restore_best_weights=True
                )


In [ ]:
history = model.fit(
            train_generator,shuffle=True,
            validation_data=test_generator,
            epochs=35,
            steps_per_epoch=len(train_generator),
            callbacks=[Log_callback,
            Model_Checkpoint_callback,
            Earlystop_callback]
        )

Epoch 1/35
 11/221 [>.............................] - ETA: 2:11:24 - loss: 9.1634

---

In [ ]:
# train_count = 1
# items_left = len(dirs)

# x_train,x_test,y_train,y_test = image_preprocess(batch_size=1,input_size=(200,200))

In [ ]:
x_train, y_train = next(train_generator)

In [ ]:
np.array(x_train[0]).shape

In [ ]:
np.array(y_train)[2]

In [ ]:
np.array(y_train)[:,0]

In [ ]:
def predict(data):
    output_location = {}
    pred = model.predict(data)
    pred = scaler.inverse_transform(pred)
    print(pred)

    
    start = 0
    stop = 3
    for i in joints:
        output_location[i] = (pred[0][start:stop]).tolist()
        start = stop
        stop += 3
    
    print(output_location) 
    
    odata = json.dumps(output_location)
    with open("./output_location.json","w") as o:
        o.write(odata)
        o.close()


    plot_results(output_location)  
    return pred

In [ ]:
num = 100

# y_true = y_train[num]
# print("y_true",y_true)


y_pred = predict(x_train[num-1:num])
print("Y_pred",y_pred) 

# print("MSE - ", mean_squared_error(y_true=y_true,y_pred=y_pred[0]))   

fig1 = plt.figure()
ax1 = fig1.add_subplot()

ax1.imshow(x_train[num])


In [ ]:
test_loc = {}

start = 0
stop = 3
for i in joints:
    test_loc[i] = (y_train[num][start:stop]).tolist()
    start = stop
    stop += 3
    
plot_results(test_loc)